<a href="https://colab.research.google.com/github/rahulgourshetty/collegezone/blob/master/Copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    LayoutLMv2Processor,
    LayoutLMv2ImageProcessor,
    LayoutLMv2ForTokenClassification,
    TrainingArguments,
    Trainer,
    EvalPrediction
)
import numpy as np
from sklearn.metrics import classification_report

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load dataset (first 100 samples for training, next 20 for testing)
dataset = load_dataset("mathieu1256/FATURA2-invoices")
train_dataset = dataset["train"].select(range(10000))
test_dataset = dataset["train"].select(range(100, 120))

# Create a label mapping for contiguous labels.
unique_labels = sorted(set(label for sublist in train_dataset["ner_tags"] for label in sublist))
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for idx, label in enumerate(unique_labels)}
num_labels = len(unique_labels)

print("Original unique labels:", unique_labels)
print("Mapping (label2id):", label2id)
print("num_labels:", num_labels)

# Load processor & image processor
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", apply_ocr=False)
image_processor = LayoutLMv2ImageProcessor.from_pretrained("microsoft/layoutlmv2-base-uncased")

# Load LayoutLMv2 model with the correct number of labels
model = LayoutLMv2ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv2-base-uncased", num_labels=num_labels
)
model.config.label2id = label2id
model.config.id2label = id2label
model.to(device)

# Preprocessing function
def preprocess_data(example):
    image = image_processor(example["image"], return_tensors="pt")["pixel_values"][0]
    encoding = processor(
        images=[image],
        text=example["tokens"],
        boxes=example["bboxes"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
    )
    encoding = {k: v.squeeze(0) for k, v in encoding.items()}

    original_labels = example["ner_tags"]
    remapped_labels = [label2id[label] for label in original_labels]
    remapped_labels += [0] * (512 - len(remapped_labels)) if len(remapped_labels) < 512 else remapped_labels[:512]
    encoding["labels"] = torch.tensor(remapped_labels, dtype=torch.long)

    bbox = example["bboxes"]
    bbox += [[0, 0, 0, 0]] * (512 - len(bbox)) if len(bbox) < 512 else bbox[:512]
    encoding["bbox"] = torch.tensor(bbox, dtype=torch.long)

    return encoding

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_data, remove_columns=["image", "tokens", "bboxes", "id", "ner_tags"])
test_dataset = test_dataset.map(preprocess_data, remove_columns=["image", "tokens", "bboxes", "id", "ner_tags"])

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    save_steps=50,
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    evaluation_strategy="epoch",
    save_total_limit=1,
)

# Compute metrics
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=2).flatten()
    labels = p.label_ids.flatten()
    mask = labels != -100  # Ignore padded labels
    preds = preds[mask]
    labels = labels[mask]

    report = classification_report(
        labels,
        preds,
        target_names=[id2label[i] for i in range(num_labels)],
        output_dict=True
    )

    # Convert all keys in the report to strings
    report = {str(key): value for key, value in report.items()}
    return report

# Setup Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# Save the model
trainer.save_model("./saved_model")
print("✅ Training and evaluation complete! Model saved in './saved_model'")


Using device: cuda
Original unique labels: [1, 2, 3, 4, 5, 6, 10, 11, 12, 13]
Mapping (label2id): {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 10: 6, 11: 7, 12: 8, 13: 9}
num_labels: 10


Some weights of LayoutLMv2ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv2-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/features/image.py:339: UserWarning: Downcasting array dtype int64 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


KeyboardInterrupt: 

In [ ]:
!pip install pytesseract
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,873 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
!pip install torch torchvision torchaudio
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
# Install Poppler (needed by pdf2image) and the required Python packages.
!apt-get install -y poppler-utils
!pip install pdf2image pytesseract transformers datasets pandas


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 0s (444 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124973 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import torch
import pytesseract
from pdf2image import convert_from_path
from transformers import LayoutLMv2Processor, LayoutLMv2ForTokenClassification
import pandas as pd

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Define the path where your fine-tuned model is saved.
model_path = "./saved_model"

# Load the processor.
# Since the saved_model folder does not include processor configuration files,
# we load the processor from the original checkpoint.
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", apply_ocr=False)

# Load the fine-tuned model.
model = LayoutLMv2ForTokenClassification.from_pretrained(model_path)
model.to(device)
model.eval()

# The id2label mapping is stored in the model configuration.
id2label = model.config.id2label

def extract_text_and_boxes(image):
    """
    Uses pytesseract to perform OCR on a PIL image.
    Returns a list of tokens and a list of bounding boxes (scaled to 0-1000).
    """
    # Run OCR on the image.
    ocr_result = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    tokens = []
    boxes = []
    num_boxes = len(ocr_result['text'])
    width, height = image.size
    for i in range(num_boxes):
        token = ocr_result['text'][i].strip()
        if token:  # Only consider non-empty tokens.
            tokens.append(token)
            # Get box values provided by Tesseract.
            x, y, w, h = ocr_result['left'][i], ocr_result['top'][i], ocr_result['width'][i], ocr_result['height'][i]
            # Convert to [x1, y1, x2, y2].
            x1, y1, x2, y2 = x, y, x + w, y + h
            # Scale the coordinates to the range 0-1000 (LayoutLMv2 expects normalized coordinates).
            x1 = int(1000 * (x1 / width))
            y1 = int(1000 * (y1 / height))
            x2 = int(1000 * (x2 / width))
            y2 = int(1000 * (y2 / height))
            boxes.append([x1, y1, x2, y2])
    return tokens, boxes

def process_pdf(pdf_path):
    """
    Processes a PDF file page by page:
      1. Converts each page to an image.
      2. Uses OCR to extract tokens and bounding boxes.
      3. Uses the LayoutLMv2 processor and model to predict token labels.
      4. Aggregates tokens by label (ignoring those with label "O").
    Returns a list of dictionaries (one per page) containing the extracted fields.
    """
    # Convert PDF pages to images.
    pages = convert_from_path(pdf_path)
    extracted_data_list = []

    for page in pages:
        # Extract tokens and bounding boxes from the image using OCR.
        tokens, boxes = extract_text_and_boxes(page)

        # Process the image and OCR output with the processor.
        # Note: We pass a list of tokens and boxes (wrapped in another list) because the processor expects a batch.
        encoding = processor(
            images=[page],
            text=[tokens],
            boxes=[boxes],
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )
        # Move tensors to the same device as the model.
        encoding = {k: v.to(device) for k, v in encoding.items()}

        # Run the model to get predictions.
        with torch.no_grad():
            outputs = model(**encoding)
        # Get predictions (logits) and convert to label IDs.
        preds = outputs.logits.argmax(-1).squeeze(0).tolist()

        # The first token is often a special token ([CLS]) and the last ([SEP]), so we assume the predictions
        # for the first len(tokens) tokens are valid.
        predicted_labels = [id2label.get(p, "O") for p in preds][1:len(tokens)+1]

        # Aggregate tokens by their predicted label (skip those with label "O").
        extracted = {}
        for token, label in zip(tokens, predicted_labels):
            if label != "O":
                if label in extracted:
                    extracted[label] += " " + token
                else:
                    extracted[label] = token
        extracted_data_list.append(extracted)

    return extracted_data_list

# --- Main Execution ---

# Provide the path to your PDF file.
pdf_path = "/content/sample.pdf"  # Replace with your PDF file path

# Process the PDF and extract structured data.
extracted_data_list = process_pdf(pdf_path)

# Save the extracted data to a CSV file (one row per PDF page).
df = pd.DataFrame(extracted_data_list)
df.to_csv("extracted_data.csv", index=False)
print("Data extracted and saved to extracted_data.csv")


Using device: cuda
Data extracted and saved to extracted_data.csv
